In [1]:
from pywhispercpp.model import Model

In [2]:
model = Model(model='large-v3', models_dir='./whisper.cpp/models')

whisper_init_from_file_with_params_no_state: loading model from '/Users/quentin/dev/podcast_audio_extractor/whisper.cpp/models/ggml-large-v3.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 0
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
whisper_init_with_params_no_state: devices    = 3
whisper_init_with_params_no_state: backends   = 3
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51866
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 1280
whisper_model_load: n_audio_head  = 20
whisper_model_load: n_audio_layer = 32
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 1280
whisper_model_load: n_text_head   = 20
whisper_model_load: n_text_layer  = 32
whisper_model_load: n_mels        = 128
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 5 (larg

In [3]:
segments = model.transcribe('./s10e43_evenMoreTrimmed_benchmark.mp3', language='fr',)

for segment in segments:
    print(segment.text)

Progress:   0%
Progress:  18%
Progress:  38%
Progress:  59%
Progress:  80%


Message publicitaire.
Le médicament Daflon 500 mg, indiqué pour soulager les jambes lourdes et douloureuses dues à l'insuffisance veineuse, agit en renforçant le tonus veineux et en protégeant les vaisseaux sanguins.
Son efficacité a été cliniquement démontrée dans le traitement des troubles de la circulation veineuse, jambes lourdes, douleurs, impatience, en complément des mesures hygiéno-diététiques.
Rendez-vous sur Daflon.fr.
Daflon 500 mg, composé de fractions flavonoïques purifiées micronisées et réservées à l'adulte, est disponible en pharmacie sans ordonnance.
Tout médicament peut exposer à des risques. Parlez-en à votre pharmacien et lisez attentivement la notice.
Si les symptômes persistent, consultez votre médecin.
Bonjour, je suis là. Vous qui n'avez pas d'invité pour nous mettre la honte de cette intro.
Bonjour, bonsoir et bienvenue dans ce nouvel épisode et dernier.
Ah, je le dis quand même.
Parce que je me disais, est-ce qu'il va faire le truc de bienvenue dans ce dernier

Progress: 100%
